In [3]:
from itertools import combinations
from binance.helpers import round_step_size
from binance.client import Client
from binance.enums import *


from datetime import datetime, timedelta, date, time
from itertools import product
from time import strftime
import scipy.stats
import pandas as pd
import numpy as np
import itertools
import warnings
import os.path
import time
import copy
import json
import os
import ta
import ast


import importlib.util
warnings.filterwarnings("ignore")

In [4]:
PATH_CONDITIONS = r'/Users/westhomas/Desktop/ENKI/ENKI_TECHNICAL_ANALYSIS_DEVELOPMENT/_CONDITIONS/'
PATH_VARIABLES  = r'/Users/westhomas/Desktop/ENKI/ENKI_TECHNICAL_ANALYSIS_DEVELOPMENT/_VARIABLES_/'

PATH_CLEAN      = r'/Users/westhomas/Desktop/ENKI/ENKI_TECHNICAL_ANALYSIS_DEVELOPMENT/A__RAW_DATA_CLEAN/'
PATH_TECHNICAL  = r'/Users/westhomas/Desktop/ENKI/ENKI_TECHNICAL_ANALYSIS_DEVELOPMENT/B__RAW_DATA_TECHNICAL/'
PATH_STRATEGIES = r'/Users/westhomas/Desktop/ENKI/ENKI_TECHNICAL_ANALYSIS_DEVELOPMENT/C__RAW_DATA_STRATEGIES/'
PATH_OUTPUT     = r'/Users/westhomas/Desktop/ENKI/ENKI_TECHNICAL_ANALYSIS_DEVELOPMENT/D__RAW_DATA_OUTPUT/'

In [5]:
class READ_IN_DATA():

    def __init__(self, COINS, TIMEFRAME, PATH):
        self.COINS                              = COINS
        self.TIMEFRAME                          = TIMEFRAME
        self.PATH                               = PATH
        self.LIST                               = self.READ_IN_RAW_DATA()


    def READ_IN_RAW_DATA(self):
        
        COIN_HIERARCHY = []
        for COIN_USE in self.COINS:

            TIME_FRAME_HIERARCHY = []
            for TIMEFRAME_USE in self.TIMEFRAME:

                FILENAME                    = f'{COIN_USE} -- {TIMEFRAME_USE}.csv'
                DF                          = (pd.read_csv((self.PATH + FILENAME))).iloc[:-1]
                DF["DATE"]                  = pd.to_datetime(DF["DATE"])


                TIME_FRAME_HIERARCHY.append(DF)
            COIN_HIERARCHY.append(TIME_FRAME_HIERARCHY)

        return COIN_HIERARCHY







class EXPORT_RAW_DATA():

    def __init__(self, DF, FILEPATH):
        self.DF                     = DF
        self.FILEPATH               = FILEPATH
        self.DF                     = self.READ_IN_RAW_DATA()

    def READ_IN_RAW_DATA(self):

        self.DF.to_csv(self.FILEPATH, index=False)
        print(f"Data has been exported to - {self.FILEPATH}")

        return self.DF









def LIST_COMBINATIONS_GROUPING(LIST_OF_LISTS, PATH, FILENAME, MAX_LENGTH):


    for i, USEFUL_STRATEGIES_LIST in enumerate(LIST_OF_LISTS):

        TEXT_FILE_PATH                  = PATH + f'{FILENAME}{i}.txt'
        BATCH_SIZE, COMBINATION_BUFFER  = 50000, []
        ALL_ELEMENTS                    = list(itertools.chain(*USEFUL_STRATEGIES_LIST.values()))
        ELEMENTS_TO_CATEGORY            = {ELEM: KEY for KEY, ELEMENTS in USEFUL_STRATEGIES_LIST.items() for ELEM in ELEMENTS}


        with open(TEXT_FILE_PATH, 'w') as FILE:
            for R in range(1, min(MAX_LENGTH, len(ALL_ELEMENTS) + 1)):
                for COMBO in itertools.combinations(ALL_ELEMENTS, R):

                    CAT_IN_COMBO = {ELEMENTS_TO_CATEGORY[ELEM] for ELEM in COMBO}

                    if len(CAT_IN_COMBO) == len(COMBO): COMBINATION_BUFFER.append(f"{list(COMBO)}\n")
                    if len(COMBINATION_BUFFER) >= BATCH_SIZE:
                        FILE.writelines(COMBINATION_BUFFER)
                        COMBINATION_BUFFER = [] 

            if COMBINATION_BUFFER: FILE.writelines(COMBINATION_BUFFER)
            
    return COMBINATION_BUFFER







def READ_IN_FILES(PATH, LIST_OF_LISTS_0):

    LIST_STRATEGIES_COMBO   = []

    for i, USEFUL_STRATEGIES_LIST in enumerate(LIST_OF_LISTS_0): 
        with open((PATH + f'VARIABLES_{i}.txt'), "r") as FILE:   LIST_STRATEGIES_COMBO.append([ast.literal_eval(LINE.strip()) for LINE in FILE])

    FLATTENED_STRATEGIES        = list(itertools.chain(*LIST_STRATEGIES_COMBO))
    UNIQUE_STRATEGIES, SEEN     = [], set() 

    for STRATEGY in FLATTENED_STRATEGIES:
        STRATEGY_TUPLE = tuple(STRATEGY)  
        if STRATEGY_TUPLE not in SEEN:
            SEEN.add(STRATEGY_TUPLE)
            UNIQUE_STRATEGIES.append(STRATEGY)


    return UNIQUE_STRATEGIES













In [6]:


def STRATEGY_GROUPINGS(PATH, DATA_DF):

    STRATEGY_COLUMNS                = [COL for COL in DATA_DF.columns if COL.startswith("STRATEGY_") 
                                        and not COL.startswith(('STRATEGY_EMA_', 'STRATEGY_SMA_', 'STRATEGY_PREV_GAIN_DOWN', 'STRATEGY_PREV_GAIN_UP', 'STRATEGY_PRICE_QUANTILE_', 'STRATEGY_Z_SCORE_ANOMALY_', 
                                                                'STRATEGY_PRICE_Z_SCORE_', 'STRATEGY_RVOL_', 'STRATEGY_ROC_', 'STRATEGY_CCI_', 'STRATEGY_WILLIAMS_', 'STRATEGY_MFI_', 'STRATEGY_MACD_', 'STRATEGY_BB',
                                                                'STRATEGY_STOCHASTIC_', 'STRATEGY_STOCHASTIC_CROSSOVER_', 'STRATEGY_STOCHASTIC_COMBINED_', 'STRATEGY_RSI_', 'STRATEGY_AVG_VOLATILITY_', 'STRATEGY_ENTROPY_', 'STRATEGY_ADX', 'STRATEGY_VWAP'))]


    STRATEGY_EMA_                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_EMA_')] 
    STRATEGY_PRICE_QUANTILE_        = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_PRICE_QUANTILE_')] 
    STRATEGY_PRICE_Z_SCORE_         = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_PRICE_Z_SCORE_')] 
    STRATEGY_RVOL_                  = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_RVOL_')] 
    STRATEGY_CCI_                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_CCI_')] 
    STRATEGY_WILLIAMS_              = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_WILLIAMS_')] 
    STRATEGY_STOCHASTIC_            = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_STOCHASTIC_')]
    STRATEGY_RSI_                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_RSI_') and not COL.endswith(('_DIVERGENCE'))]
    STRATEGY_MACD_                  = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_MACD_')]
    STRATEGY_BB                     = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_BB')]
    STRATEGY_ADX                    = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_ADX')]
    STRATEGY_VWAP                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_VWAP')]



    ###### 1. Trend-Following + Momentum ######
    LIST_1  = {'STRATEGY_EMA_'              : STRATEGY_EMA_,           #   Why: EMA confirms the trend, MACD confirms momentum, and RSI avoids overbought/oversold trades.
                'STRATEGY_MACD_'            : STRATEGY_MACD_,
                'STRATEGY_RSI_'             : STRATEGY_RSI_}


    LIST_2  = {'STRATEGY_ADX'               : STRATEGY_ADX,
                'STRATEGY_WILLIAMS_'        : STRATEGY_WILLIAMS_,
                'STRATEGY_STOCHASTIC_'      : STRATEGY_STOCHASTIC_}


    ###### 2. Volatility + Trend ######
    LIST_3  = {'STRATEGY_BB'                : STRATEGY_BB,               #   Why: CCI confirms cyclical movements alongside volatility-based breakout indicators.
                'STRATEGY_CCI_'             : STRATEGY_CCI_,
                'STRATEGY_PRICE_Z_SCORE_'   : STRATEGY_PRICE_Z_SCORE_}

    LIST_4  = {'STRATEGY_EMA_'              : STRATEGY_EMA_,             #   Why: VWAP and EMA detect price levels, and RSI avoids overbought/oversold trades.
                'STRATEGY_VWAP'             : STRATEGY_VWAP,
                'STRATEGY_RSI_'             : STRATEGY_RSI_}

    ###### 3. Momentum + Volatility ######
    LIST_5  = {'STRATEGY_RSI_'              : STRATEGY_RSI_,           #   Why: RSI measures momentum, and RVOL highlights periods of high trading activity, confirming momentum strength.
                'STRATEGY_RVOL_'            : STRATEGY_RVOL_,
                'STRATEGY_PRICE_Z_SCORE_'   : STRATEGY_PRICE_Z_SCORE_}


    LIST_7  = {'STRATEGY_CCI_'              : STRATEGY_CCI_,        #   Why: VWAP and CCI detect price cyclicality and conditions, while RSI avoids weak momentum trades.
                'STRATEGY_VWAP'             : STRATEGY_VWAP,
                'STRATEGY_RSI_'             : STRATEGY_RSI_}


    ###### 4. Overbought/Oversold + Divergence ######
    LIST_8  = {'STRATEGY_RSI_'              : STRATEGY_RSI_,                #   Why: RSI spots overbought/oversold levels, and MACD divergence confirms reversals.
                'STRATEGY_MACD_'            : STRATEGY_MACD_,
                'STRATEGY_BB'               : STRATEGY_BB}


    ###### 5. Pattern Recognition + Volatility ######
    LIST_10 = {'STRATEGY_COLUMNS'           : STRATEGY_COLUMNS,              #   Why: Double bottoms indicate reversals, and RVOL confirms accumulation phases or breakout volumes.
                'STRATEGY_RVOL_'            : STRATEGY_RVOL_,
                'STRATEGY_BB'               : STRATEGY_BB,
                'STRATEGY_PRICE_Z_SCORE_'   : STRATEGY_PRICE_Z_SCORE_,
                'STRATEGY_PRICE_QUANTILE_'  : STRATEGY_PRICE_QUANTILE_}


    FILENAME            = 'VARIABLES_'
    LIST_OF_LISTS       = [LIST_1, LIST_2, LIST_3, LIST_4,LIST_5, LIST_7, LIST_8, LIST_10]
    MAX_LENGTH          = 5 if len(LIST_OF_LISTS) > 2 else 25
    OUTPUT              = LIST_COMBINATIONS_GROUPING(LIST_OF_LISTS, PATH, FILENAME, MAX_LENGTH)
    UNIQUE_STRATEGIES   = READ_IN_FILES(PATH, LIST_OF_LISTS)
    
    return UNIQUE_STRATEGIES







In [7]:

def STRATEGY_DEVELOPMENT(DF):

    LIST        = [4, 8, 12, 18, 24, 36, 50, 100, 200]
    PAIR_COMBO  = list(combinations(LIST, 2))
    ROC_OPTIONS = [0.5, 1, 2, 5]
    RSI_OPTIONS = [10, 20, 30, 40]
    RSI_LIST    = [14, 20, 30]
    PERIODS     = [7, 10, 20]
    CCI_OPTIONS = [1, 2]
    FIELDS      = ['EMA']


    for FIELD in FIELDS:
        for I in range(len(PAIR_COMBO)):
            LOWER                                   = f'{FIELD}_{PAIR_COMBO[I][0] if PAIR_COMBO[I][0] < PAIR_COMBO[I][1] else PAIR_COMBO[I][1]}'
            UPPER                                   = f'{FIELD}_{PAIR_COMBO[I][1] if PAIR_COMBO[I][0] < PAIR_COMBO[I][1] else PAIR_COMBO[I][0]}'
            DF[f'STRATEGY_{LOWER}__{UPPER}']        = np.where((DF[LOWER].shift(1) < DF[UPPER].shift(1)) & (DF[LOWER] > DF[UPPER]), 'LONG', np.where((DF[LOWER].shift(1) > DF[UPPER].shift(1)) & (DF[LOWER] < DF[UPPER]), 'SHORT','STATIC'))


    for I in range(len(RSI_LIST)):
        for G in range(len(RSI_OPTIONS)):
            RSI_LOWER                               = RSI_OPTIONS[G]
            RSI_UPPER                               = 100 - RSI_OPTIONS[G]
            RSI_COLUMN                              = f'RSI_{RSI_LIST[I]}'

            DF[f'STRATEGY_RSI_{RSI_LIST[I]}__{G}']  = np.where(DF[RSI_COLUMN] < RSI_LOWER, 'LONG',np.where(DF[RSI_COLUMN] > RSI_UPPER, 'SHORT', 'STATIC'))


    DF['STRATEGY_MACD_SIGNAL_LINE']                 = np.where( (DF['MACD'] > DF['MACD_SIGNAL']), 'LONG',np.where(DF['MACD'] < DF['MACD_SIGNAL'], 'SHORT', 'STATIC'))
    DF['STRATEGY_MACD_ZERO_LINE_STRATEGY']          = np.where(DF['MACD'] > 0, 'LONG',np.where(DF['MACD'] < 0, 'SHORT', 'STATIC'))
    DF['STRATEGY_MACD_DIFF_STRATEGY']               = np.where(DF['MACD_DIFF'] > 0, 'LONG',np.where(DF['MACD_DIFF'] < 0, 'SHORT', 'STATIC'))
    DF['STRATEGY_MACD_COMBINED_STRATEGY']           = np.where((DF['MACD'] > DF['MACD_SIGNAL']) & (DF['MACD'] > 0) & (DF['MACD_DIFF'] > 0), 'LONG',np.where((DF['MACD'] < DF['MACD_SIGNAL']) & (DF['MACD'] < 0) & (DF['MACD_DIFF'] < 0), 'SHORT','STATIC'))
    DF['STRATEGY_BB_MEAN_REVERSION']                = np.where(DF['CLOSE'].shift(1) <= DF['BB_LOWER'], 'LONG',np.where(DF['CLOSE'].shift(1) >= DF['BB_UPPER'], 'SHORT', 'STATIC'))
    DF['STRATEGY_BB_BREAKOUT']                      = np.where((DF['CLOSE'].shift(1) > DF['BB_UPPER']) & (DF['BB_WIDTH'] > DF['BB_WIDTH'].rolling(20).mean()), 'LONG',np.where((DF['CLOSE'].shift(1) < DF['BB_LOWER']) & (DF['BB_WIDTH'] > DF['BB_WIDTH'].rolling(20).mean()), 'SHORT','STATIC'))
    DF['STRATEGY_BB_MIDDLE_REVERSION']              = np.where((DF['CLOSE'].shift(2) < DF['BB_MIDDLE']) & (DF['CLOSE'].shift(1) > DF['BB_MIDDLE']), 'LONG',np.where((DF['CLOSE'].shift(2) > DF['BB_MIDDLE']) & (DF['CLOSE'].shift(1) < DF['BB_MIDDLE']), 'SHORT','STATIC'))
    DF['STRATEGY_VWAP']                             = np.where(DF['CLOSE'].shift(1) > DF['VWAP'], 'LONG',np.where(DF['CLOSE'].shift(1) < DF['VWAP'], 'SHORT', 'STATIC'))
    DF['STRATEGY_PSAR']                             = np.where(DF['CLOSE'].shift(1) > DF['PSAR'], 'LONG',np.where(DF['CLOSE'].shift(1) < DF['PSAR'], 'SHORT', 'STATIC'))
    DF['STRATEGY_TSI']                              = np.where(DF['TSI'] > 0, 'LONG',np.where(DF['TSI'] < 0, 'SHORT', 'STATIC'))
    DF['STRATEGY_ADX']                              = np.where((DF['ADX_POS'] > DF['ADX_NEG']) & (DF['ADX'] > DF['ADX'].shift(1)), 'LONG',np.where((DF['ADX_NEG'] > DF['ADX_POS']) & (DF['ADX'] > DF['ADX'].shift(1)), 'SHORT','STATIC'))

    for G in range(len(RSI_OPTIONS)):
        LOWER                                       = RSI_OPTIONS[G]
        UPPER                                       = 100 - RSI_OPTIONS[G]
        DF[f'STRATEGY_WILLIAMS_%R_{G}']             = np.where(DF['WILLIAMS_%R'] < (-1*UPPER), 'LONG',np.where(DF['WILLIAMS_%R'] > (-1*LOWER), 'SHORT', 'STATIC'))
        DF[f'STRATEGY_MFI_{G}']                     = np.where(DF['MFI'] < LOWER, 'LONG',np.where(DF['MFI'] > UPPER, 'SHORT', 'STATIC'))
        DF[f'STRATEGY_STOCHASTIC_{G}']              = np.where((DF['%K'] < LOWER) & (DF['%K'] > DF['%K'].shift(1)), 'LONG',  np.where((DF['%K'] > UPPER) & (DF['%K'] < DF['%K'].shift(1)), 'SHORT', 'STATIC'))
        DF[f'STRATEGY_STOCHASTIC_CROSSOVER_{G}']    = np.where((DF['%K'].shift(1) < DF['%D'].shift(1)) & (DF['%K'] > DF['%D']), 'LONG', np.where((DF['%K'].shift(1) > DF['%D'].shift(1)) & (DF['%K'] < DF['%D']), 'SHORT', 'STATIC'))
        DF[f'STRATEGY_STOCHASTIC_COMBINED_{G}']     = np.where(((DF['%K'] < LOWER) & (DF['%K'] > DF['%K'].shift(1))) | ((DF['%K'].shift(1) < DF['%D'].shift(1)) & (DF['%K'] > DF['%D'])), 'LONG',
                                                        np.where(((DF['%K'] > UPPER) & (DF['%K'] < DF['%K'].shift(1))) | ((DF['%K'].shift(1) > DF['%D'].shift(1)) & (DF['%K'] < DF['%D'])), 'SHORT','STATIC'))

    for G in range(len(CCI_OPTIONS)):               DF[f'STRATEGY_CCI_{G}'] = np.where(DF['CCI'] < (-1*(100 / CCI_OPTIONS[G])), 'LONG',np.where(DF['CCI'] > ((100 / CCI_OPTIONS[G])), 'SHORT', 'STATIC'))
    for G in range(len(ROC_OPTIONS)):               DF[f'STRATEGY_ROC_{G}'] = np.where(DF['ROC'] > ROC_OPTIONS[G], 'LONG',np.where(DF['ROC'] < (-1*ROC_OPTIONS[G]), 'SHORT', 'STATIC'))

    DF['STRATEGY_HAMMER']                           = np.where(DF['HAMMER'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_INV_HEAD_SHOULDERS']               = np.where(DF['INV_HEAD_SHOULDERS'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_DOUBLE_BOTTOM']                    = np.where(DF['DOUBLE_BOTTOM'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_BULL_FLAG']                        = np.where(DF['BULL_FLAG'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_ASC_TRIANGLE']                     = np.where(DF['ASC_TRIANGLE'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_BULL_ENGULF']                      = np.where(DF['BULL_ENGULF'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_HEAD_SHOULDERS']                   = np.where(DF['HEAD_SHOULDERS'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_DOUBLE_TOP']                       = np.where(DF['DOUBLE_TOP'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_BEAR_FLAG']                        = np.where(DF['BEAR_FLAG'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_DESC_TRIANGLE']                    = np.where(DF['DESC_TRIANGLE'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_SHOOTING_STAR']                    = np.where(DF['SHOOTING_STAR'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_BEAR_ENGULF']                      = np.where(DF['BEAR_ENGULF'] == 1, 'SHORT', 'STATIC')

    for G in range(len(PERIODS)):
        CHOSEN = PERIODS[G]
        DF[f'STRATEGY_RVOL_{G}']                         = np.where(DF[f'RVOL_{CHOSEN}'] > 1, 'LONG',np.where(DF[f'RVOL_{CHOSEN}'] < 1, 'SHORT', 'STATIC'))
        DF[f'STRATEGY_PRICE_Z_SCORE_{G}']                = np.where(DF[f'PRICE_Z_SCORE_{CHOSEN}'] < -2, 'LONG',np.where(DF[f'PRICE_Z_SCORE_{CHOSEN}'] > 2, 'SHORT', 'STATIC'))

        for H in range(len(RSI_OPTIONS)):
            LOWER                                       = RSI_OPTIONS[H]/100
            UPPER                                       = 1 - (RSI_OPTIONS[H]/100)
            DF[f'STRATEGY_PRICE_QUANTILE_{G}_{H}']      = np.where(DF[f'PRICE_QUANTILE_{CHOSEN}'] < LOWER, 'LONG',np.where(DF[f'PRICE_QUANTILE_{CHOSEN}'] > UPPER, 'SHORT', 'STATIC'))


    return DF








TIMEFRAME               = ['1D']
COIN                    = ['BTCUSDT']
DATA                    = READ_IN_DATA(COIN, TIMEFRAME, PATH_TECHNICAL)
DATA_DF                 = DATA.LIST[0][0].copy()
STRATEGY_DF             = STRATEGY_DEVELOPMENT(DATA_DF)
UNIQUE_STRATEGIES       = STRATEGY_GROUPINGS(PATH_VARIABLES, STRATEGY_DF)
len(UNIQUE_STRATEGIES)





11459

In [8]:


class BACKTEST_NEW():
    def __init__(self, INPUT_LIST, START_DATE, LEVERAGE, PERCENTAGE_USED, STARTING_BALANCE):
        self.INPUT_LIST         = INPUT_LIST
        self.START_DATE         = START_DATE
        self.LEVERAGE           = LEVERAGE
        self.PERCENTAGE_USED    = PERCENTAGE_USED
        self.STARTING_BALANCE   = STARTING_BALANCE
        self.API_DATA           = self.BACKTEST()

    def BACKTEST(self):
        DF_UPPER                    = self.INPUT_LIST[1]
        DF_LOWER                    = self.INPUT_LIST[0]
        self.START_DATE             = pd.to_datetime(self.START_DATE)
        DF_UPPER                    = DF_UPPER[DF_UPPER['DATE'] >= self.START_DATE].reset_index(drop=True)
        DF_UPPER['STR']             = (DF_UPPER['DATE'].dt.date).astype("string") + ' ' + (DF_UPPER['DATE'].dt.time).astype("string")
        DF_LOWER['STR']             = (DF_LOWER['DATE'].dt.date).astype("string") + ' ' + (DF_LOWER['DATE'].dt.time).astype("string")
        DATA_MAIN                   = DF_UPPER[['STR', 'SIGNAL', 'TP', 'SL']]
        FINAL_DF                    = DF_LOWER.merge(DATA_MAIN, how='left', on=['STR'])
        FINAL_DF['SIGNAL']          = FINAL_DF['SIGNAL'].fillna('STATIC')
        FINAL_DF['TRADE_STATUS']    = None 

        TRADES_TP_SL                = self.RUN_BACKTEST(FINAL_DF)
        TRADES_NEXT_TRADE           = self.EXIT_AT_NEXT_TRADE(FINAL_DF)
        TRADES_COMBINED             = self.COMBINED(TRADES_TP_SL, TRADES_NEXT_TRADE)
        SCENARIOS                   = ['TP_SL_ONLY', 'LEADING_ONLY', 'TP_SL_LEADING']
        SCENARIOS_VARIABLES         = [TRADES_TP_SL, TRADES_NEXT_TRADE, TRADES_COMBINED]
        TIME_LIST                   = []

        for A in range(len(SCENARIOS)): 
            if A == 0:              

                SCENARIOS_VARIABLES[A]  = self.REMOVE_OVERLAPPING_TRADES(SCENARIOS_VARIABLES[A])
                PORTFOLIO_RESULTS       = self.PORTFOLIO_TIMELINE(SCENARIOS_VARIABLES[A])
                OVERVIEW                = self.OVERVIEW(SCENARIOS_VARIABLES[A], SCENARIOS[A])
                OVERVIEW_DF             = OVERVIEW.copy()

            else:
            
                PORTFOLIO_RESULTS       = self.PORTFOLIO_TIMELINE(SCENARIOS_VARIABLES[A])
                OVERVIEW                = self.OVERVIEW(SCENARIOS_VARIABLES[A], SCENARIOS[A])             
                OVERVIEW_DF             = pd.concat([OVERVIEW_DF, OVERVIEW], ignore_index=True)


            TIME_LIST.append(PORTFOLIO_RESULTS)

        return OVERVIEW_DF  #[OVERVIEW_DF, TIME_LIST, SCENARIOS_VARIABLES, FINAL_DF]


    def COMBINED(self, TRADES_TP_SL, TRADES_NEXT_TRADE):

        TRADES_COMBINED                         = TRADES_TP_SL.merge(TRADES_NEXT_TRADE, on="START_INDEX", how="inner")
        TRADES_COMBINED['END_INDEX']            = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['END_INDEX_x'], TRADES_COMBINED['END_INDEX_y'])
        TRADES_COMBINED['STATUS']               = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['STATUS_x'], TRADES_COMBINED['STATUS_y'])   
        TRADES_COMBINED['TIME_COMPLETED']       = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['TIME_COMPLETED_x'], TRADES_COMBINED['TIME_COMPLETED_y'])   
        TRADES_COMBINED['PERCENTAGE_CHANGE']    = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['PERCENTAGE_CHANGE_x'], TRADES_COMBINED['PERCENTAGE_CHANGE_y'])  
        TRADES_COMBINED                         = TRADES_COMBINED[['START_INDEX', 'END_INDEX', 'STATUS', 'TIME_COMPLETED', 'PERCENTAGE_CHANGE']]

        return TRADES_COMBINED


    def RUN_BACKTEST(self, FINAL_DF):
        RESULTS, ACTIVE_TRADE = [], None

        for IDX, ROW in FINAL_DF.iterrows():
            if ACTIVE_TRADE is None and ROW['SIGNAL'] != "STATIC":
                ACTIVE_TRADE = {"INDEX": IDX, "TRADE": ROW}

            if ACTIVE_TRADE:
                OUTCOME, COMPLETED_TIME, COMPLETED_INDEX, PERCENTAGE_CHANGE = self.TRADE_ANALYSIS(ACTIVE_TRADE["TRADE"], FINAL_DF.iloc[IDX:])

                if OUTCOME in ["SUCCESS", "FAILURE"]:
                    RESULTS.append({"START_INDEX"       : ACTIVE_TRADE["INDEX"],
                                    "END_INDEX"         : COMPLETED_INDEX,
                                    "STATUS"            : OUTCOME,
                                    "TIME_COMPLETED"    : COMPLETED_TIME,
                                    "PERCENTAGE_CHANGE" : PERCENTAGE_CHANGE})

                    FINAL_DF.at[ACTIVE_TRADE["INDEX"], 'TRADE_STATUS'] = OUTCOME
                    ACTIVE_TRADE = None  

        return pd.DataFrame(RESULTS)




    def TRADE_ANALYSIS(self, ROW, TRADE_DATA):
        DIRECTION, TRADE_START, TRADE_OPEN      = ROW['SIGNAL'], ROW['DATE'], ROW['OPEN']
        TRADE_TP, TRADE_SL                      = ROW['TP'], ROW['SL']
        if TRADE_DATA.empty:                    return "FAILURE", TRADE_START, TRADE_DATA.index[-1], 0

        if DIRECTION == "LONG":
            TP_IDX = TRADE_DATA.loc[TRADE_DATA['HIGH'] >= TRADE_TP, 'DATE'].first_valid_index()
            SL_IDX = TRADE_DATA.loc[TRADE_DATA['LOW'] <= TRADE_SL, 'DATE'].first_valid_index()
        else:  
            TP_IDX = TRADE_DATA.loc[TRADE_DATA['LOW'] <= TRADE_TP, 'DATE'].first_valid_index()
            SL_IDX = TRADE_DATA.loc[TRADE_DATA['HIGH'] >= TRADE_SL, 'DATE'].first_valid_index()

        TP_HIT = TRADE_DATA.at[TP_IDX, 'DATE'] if TP_IDX in TRADE_DATA.index else None
        SL_HIT = TRADE_DATA.at[SL_IDX, 'DATE'] if SL_IDX in TRADE_DATA.index else None

        if TP_IDX is not None and SL_IDX is not None:       return ("SUCCESS", TP_HIT, TP_IDX, abs((TRADE_TP - TRADE_OPEN) / TRADE_OPEN) * 100) if TP_IDX < SL_IDX else ("FAILURE", SL_HIT, SL_IDX, -abs((TRADE_SL - TRADE_OPEN) / TRADE_OPEN) * 100)
        elif TP_IDX is not None:                            return "SUCCESS", TP_HIT, TP_IDX, abs((TRADE_TP - TRADE_OPEN) / TRADE_OPEN) * 100
        elif SL_IDX is not None:                            return "FAILURE", SL_HIT, SL_IDX, -abs((TRADE_SL - TRADE_OPEN) / TRADE_OPEN) * 100

        return "FAILURE", TRADE_DATA['DATE'].iloc[-1], TRADE_DATA.index[-1], 0



    def EXIT_AT_NEXT_TRADE(self, FINAL_DF):

        TRADE_DF                = (FINAL_DF[FINAL_DF['SIGNAL'] != "STATIC"])
        TRADE_DF["END_INDEX"]  = TRADE_DF.index.to_series().shift(-1)
        TRADE_DF["EXIT_DATE"]   = TRADE_DF["DATE"].shift(-1)
        TRADE_DF["EXIT_PRICE"]  = TRADE_DF["OPEN"].shift(-1)

        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("END_INDEX")]   = FINAL_DF.index[-1]
        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("EXIT_DATE")]    = FINAL_DF["DATE"].iloc[-1]
        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("EXIT_PRICE")]   = FINAL_DF["CLOSE"].iloc[-1]

        TRADE_DF["STATUS"]              = np.where(((TRADE_DF["EXIT_PRICE"] > TRADE_DF["OPEN"]) & (TRADE_DF["SIGNAL"] == "LONG")) | ((TRADE_DF["EXIT_PRICE"] < TRADE_DF["OPEN"]) & (TRADE_DF["SIGNAL"] == "SHORT")),  "SUCCESS", "FAILURE")
        TRADE_DF["PERCENTAGE_CHANGE"]   = np.where(TRADE_DF["SIGNAL"] == "LONG", ((TRADE_DF["EXIT_PRICE"] - TRADE_DF["OPEN"]) / TRADE_DF["OPEN"]) * 100, ((TRADE_DF["OPEN"] - TRADE_DF["EXIT_PRICE"]) / TRADE_DF["OPEN"]) * 100)

        return TRADE_DF[["END_INDEX", "STATUS", "EXIT_DATE", "PERCENTAGE_CHANGE"]].reset_index().rename(columns={"index": "START_INDEX", "EXIT_DATE": "TIME_COMPLETED"})




    def PORTFOLIO_TIMELINE(self, INPUT_DF):


        BALANCE, N                      = self.STARTING_BALANCE, len(INPUT_DF)
        FEE_RATE, ALLOCATION            = 0.00075, 0.985 if self.PERCENTAGE_USED == 1 else self.PERCENTAGE_USED
        if INPUT_DF.empty:              return pd.DataFrame(columns=["START_AMOUNT", "TRADE_AMOUNT", "LEVERED_AMOUNT", "PERCENTAGE_CHANGE", "GROSS_PROFIT", "FEES", "NET_PROFIT", "END_AMOUNT"])

        start_amounts                   = np.zeros(N)
        trade_amounts                   = np.zeros(N)
        levered_amounts                 = np.zeros(N)
        gross_profits                   = np.zeros(N)
        fees                            = np.zeros(N)
        net_profits                     = np.zeros(N)
        end_amounts                     = np.zeros(N)
        start_amounts[0]                = BALANCE 
        percentage_changes              = INPUT_DF["PERCENTAGE_CHANGE"].values / 100  
        zero_balance_index              = None  

        for i in range(N):
            if zero_balance_index is not None: start_amounts[i] = trade_amounts[i] = levered_amounts[i] = gross_profits[i] = fees[i] = net_profits[i] = end_amounts[i] = 0
            else:
                trade_amounts[i]        = start_amounts[i] * ALLOCATION
                levered_amounts[i]      = trade_amounts[i] * self.LEVERAGE
                gross_profits[i]        = levered_amounts[i] * percentage_changes[i]
                fees[i]                 = levered_amounts[i] * FEE_RATE
                net_profits[i]          = gross_profits[i] - fees[i]
                end_amounts[i]          = start_amounts[i] + net_profits[i]

                if end_amounts[i] <= 0:
                    zero_balance_index  = i 
                    end_amounts[i]      = 0  

            if i < N - 1 and zero_balance_index is None:
                start_amounts[i + 1]    = end_amounts[i]

        INPUT_DF["START_AMOUNT"]        = start_amounts
        INPUT_DF["TRADE_AMOUNT"]        = trade_amounts
        INPUT_DF["LEVERED_AMOUNT"]      = levered_amounts
        INPUT_DF["GROSS_PROFIT"]        = gross_profits
        INPUT_DF["FEES"]                = fees
        INPUT_DF["NET_PROFIT"]          = net_profits
        INPUT_DF["END_AMOUNT"]          = end_amounts

        return INPUT_DF[["START_AMOUNT", "TRADE_AMOUNT", "LEVERED_AMOUNT", "PERCENTAGE_CHANGE", "GROSS_PROFIT", "FEES", "NET_PROFIT", "END_AMOUNT"]]




    def OVERVIEW(self, RESULTS_TABLE, SCENARIO_DESC):

        if RESULTS_TABLE.empty:     return pd.DataFrame([{"METHOD": SCENARIO_DESC, "END_BALANCE": self.STARTING_BALANCE, "WIN_RATE": 0, "NBR_TRADES": 0}])
        WIN_TRADES                  = (RESULTS_TABLE["PERCENTAGE_CHANGE"] > 0).sum()
        TOTAL_TRADES                = len(RESULTS_TABLE)
        WIN_RATE                    = WIN_TRADES / TOTAL_TRADES if TOTAL_TRADES > 0 else 0

        return pd.DataFrame([{  "METHOD"        : SCENARIO_DESC,
                                "END_BALANCE"   : RESULTS_TABLE["END_AMOUNT"].iloc[-1],
                                "WIN_RATE"      : WIN_RATE,
                                "NBR_TRADES"    : TOTAL_TRADES}])
    


    def REMOVE_OVERLAPPING_TRADES(self, DF):

        DF_USE = DF.copy()
        DF_USE = DF_USE.sort_values("START_INDEX").reset_index(drop=True)
        NON_OVERLAPPING = []  
        
        for i in range(len(DF_USE)):
            if i == 0:          NON_OVERLAPPING.append(DF_USE.iloc[i]) 
            else:
                PREV_TRADE      = NON_OVERLAPPING[-1]
                CURRENT_TRADE   = DF_USE.iloc[i]

                if PREV_TRADE["END_INDEX"] > CURRENT_TRADE["START_INDEX"]:     continue  
                else:    NON_OVERLAPPING.append(CURRENT_TRADE)
        
        return pd.DataFrame(NON_OVERLAPPING)





In [9]:
import ast
import re



def DATA_PULL():
    folder_path = r'/Users/westhomas/Desktop/ENKI/ENKI_TECHNICAL_ANALYSIS_DEVELOPMENT/_CONDITIONS/'
    parsed_data = {}


    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):  
            file_path = os.path.join(folder_path, file_name)
            
            with open(file_path, "r") as file:
                content = file.read().strip()  
                try:                            parsed_data[file_name] = ast.literal_eval(content) 
                except Exception as e:          print(f"Error parsing {file_name}: {e}")

    
    TP_SL_OPTIONS   = parsed_data['TP_SL_OPTIONS.txt']
    RL_RW_OPTIONS   = parsed_data['RL_RW_OPTIONS.txt']
    STD_WINDOW      = parsed_data['STD_WINDOW.txt']
    FIB_RATIOS      = {"FIB_100" : 1.0, "FIB_1618" : 1.618, "FIB_2618" : 2.618, "FIB_4236" : 4.236}
    
    return TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS




def ANALYSIS(STATUS, TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS):

    def categorize_tp_sl(column_name):

        match                           = re.match(r"STATUS__TP_LONG_(\d+)$", column_name)
        if match:
            idx                         = int(match.group(1)) - 1
            return {"Category"          : "Fixed TP/SL",
                    "TP_Value"          : TP_SL_OPTIONS[idx][0] if idx < len(TP_SL_OPTIONS) else None,
                    "SL_Value"          : TP_SL_OPTIONS[idx][1] if idx < len(TP_SL_OPTIONS) else None}

        match                           = re.match(r"STATUS__TP_LONG_RL_(\d+)$", column_name)
        if match:
            idx                         = int(match.group(1)) - 1
            return {"Category"          : "Risk Level TP/SL",
                    "Risk_Level"        : RL_RW_OPTIONS[idx][0] if idx < len(RL_RW_OPTIONS) else None,
                    "Reward_Factor"     : RL_RW_OPTIONS[idx][1] if idx < len(RL_RW_OPTIONS) else None}

        match                           = re.match(r"STATUS__TP_LONG_RL_STD_(\d+)_(\d+)$", column_name)
        if match:
            risk_idx, std_idx           = int(match.group(1)) - 1, int(match.group(2))
            return {"Category"          : "Std Dev TP/SL",
                    "Risk_Level"        : RL_RW_OPTIONS[risk_idx][0] if risk_idx < len(RL_RW_OPTIONS) else None,
                    "Reward_Factor"     : RL_RW_OPTIONS[risk_idx][1] if risk_idx < len(RL_RW_OPTIONS) else None,
                    "Std_Window"        : STD_WINDOW[std_idx] if std_idx < len(STD_WINDOW) else None}

        match                           = re.match(r"STATUS__TP_LONG_FIB_(\d+)$", column_name)
        if match:
            fib_key                     = f"FIB_{match.group(1)}"
            return {"Category"          : "Fibonacci TP/SL",
                    "Fibonacci_Level"   : FIB_RATIOS.get(fib_key, None)}

        return {"Category"              : "Unknown",
                "Description"           : "No matching category"}


    categorized_tp_sl = {col: categorize_tp_sl(col) for col in STATUS}

    return categorized_tp_sl



TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS                = DATA_PULL()




In [10]:

STRATEGIES_GROUPS                           = []
TIMEFRAME_RD                                = ['1D', '4H']
COINS                                       = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
DATA_RDC                                    = READ_IN_DATA(COINS, ['4H'], PATH_CLEAN)
DATA_RDS                                    = READ_IN_DATA(COINS, TIMEFRAME_RD, PATH_STRATEGIES)


In [11]:
import re



T                                   = 1
C                                   = 0
STRATEGY_USE                        = 'STRATEGY_NBR_149'
TP_SL_USE                           = '27'

DATA_DF_TEST                        = DATA_RDS.LIST[C][T].copy()
DATA_DF_START                       = DATA_RDC.LIST[C][0].copy()
RETAIN_COLS                         = ["DATE", "OPEN", "HIGH", "LOW", "CLOSE", "VOLUME", 'SIGNAL', 'TP', 'SL']
CONDENSED_COLS                      = [STRATEGY_USE, f'TP_LONG_{TP_SL_USE}', f'SL_LONG_{TP_SL_USE}', f'TP_SHORT_{TP_SL_USE}', f'SL_SHORT_{TP_SL_USE}']
DATA_DF_TEST['TP']                  = np.where((DATA_DF_TEST[STRATEGY_USE] == "LONG"), DATA_DF_TEST[f'TP_LONG_{TP_SL_USE}'], DATA_DF_TEST[f'TP_SHORT_{TP_SL_USE}'])
DATA_DF_TEST['SL']                  = np.where((DATA_DF_TEST[STRATEGY_USE] == "LONG"), DATA_DF_TEST[f'SL_LONG_{TP_SL_USE}'], DATA_DF_TEST[f'SL_SHORT_{TP_SL_USE}'])
DATA_DF_TEST['SIGNAL']              = DATA_DF_TEST[STRATEGY_USE]
DATA_DF_TEST                        = DATA_DF_TEST[RETAIN_COLS]
DATA_INPUT_LIST                     = [DATA_DF_START, DATA_DF_TEST]


BACKTEST_OUTPUT                     = BACKTEST_NEW(DATA_INPUT_LIST, '2022-01-01', 5, 0.75, 100)

MATCH                               = re.search(r"STRATEGY_NBR_(\d+)", STRATEGY_USE)
STRATEGY_FIGURE                     = int(MATCH.group(1)) if MATCH else None
STRATEGY_NAME                       = UNIQUE_STRATEGIES[STRATEGY_FIGURE-1]
IMPLEMENTED_PROFIT                  = ANALYSIS([f'STATUS__TP_LONG_{TP_SL_USE}'], TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS)




print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_EMA_4__EMA_100', 'STRATEGY_MACD_SIGNAL_LINE']
{'STATUS__TP_LONG_27': {'Category': 'Fixed TP/SL', 'TP_Value': 5, 'SL_Value': 2}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,3085.004722,0.456311,103
1,LEADING_ONLY,1484.924050,0.336066,122
2,TP_SL_LEADING,2933.675753,0.426230,122


In [59]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#DOGEUSDT -- 4H	STRATEGY_NBR_4118	35	            6,338 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 3

STRATEGY_USE                        = 'STRATEGY_NBR_4118'
TP_SL_USE                           = '35'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_TSI', 'STRATEGY_RVOL_0', 'STRATEGY_BB_MIDDLE_REVERSION']
{'STATUS__TP_LONG_35': {'Category': 'Fixed TP/SL', 'TP_Value': 10, 'SL_Value': 5}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,6338.085173,0.486239,109
1,LEADING_ONLY,11567.380721,0.600000,200
2,TP_SL_LEADING,2175.552852,0.550000,200


In [64]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#BTCUSDT -- 4H	STRATEGY_NBR_149	27	            1,187 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 0

STRATEGY_USE                        = 'STRATEGY_NBR_149'
TP_SL_USE                           = '27'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_EMA_4__EMA_100', 'STRATEGY_MACD_SIGNAL_LINE']
{'STATUS__TP_LONG_27': {'Category': 'Fixed TP/SL', 'TP_Value': 5, 'SL_Value': 2}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,3085.004722,0.456311,103
1,LEADING_ONLY,1484.924050,0.336066,122
2,TP_SL_LEADING,2933.675753,0.426230,122


In [47]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#ETHUSDT -- 4H	STRATEGY_NBR_2405	18	            1,237,929 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 1

STRATEGY_USE                        = 'STRATEGY_NBR_2405'
TP_SL_USE                           = '18'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_ADX']
{'STATUS__TP_LONG_18': {'Category': 'Fixed TP/SL', 'TP_Value': 2, 'SL_Value': 10}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,1.237929e+06,0.892857,840
1,LEADING_ONLY,9.302763e-02,0.534705,3256
2,TP_SL_LEADING,3.939248e-01,0.555590,3256


In [51]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#XRPUSDT -- 4H	STRATEGY_NBR_3544	36	            706,259 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 2

STRATEGY_USE                        = 'STRATEGY_NBR_3544'
TP_SL_USE                           = '36'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_PSAR']
{'STATUS__TP_LONG_36': {'Category': 'Fixed TP/SL', 'TP_Value': 10, 'SL_Value': 10}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,7.062588e+05,0.627820,266
1,LEADING_ONLY,3.611459e-07,0.475309,6966
2,TP_SL_LEADING,1.895514e-06,0.475452,6966


In [42]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#ADAUSDT -- 4H	STRATEGY_NBR_40	    30	            107,350,457 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 4

STRATEGY_USE                        = 'STRATEGY_NBR_40'
TP_SL_USE                           = '30'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_MACD_COMBINED_STRATEGY']
{'STATUS__TP_LONG_30': {'Category': 'Fixed TP/SL', 'TP_Value': 5, 'SL_Value': 10}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,1.073505e+08,0.774955,551
1,LEADING_ONLY,1.018673e-04,0.506815,4035
2,TP_SL_LEADING,1.806849e-03,0.510781,4035


In [36]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#DOGEUSDT -- 4H	STRATEGY_NBR_40	    30	            1,387,651,315 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 3

STRATEGY_USE                        = 'STRATEGY_NBR_40'
TP_SL_USE                           = '30'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_MACD_COMBINED_STRATEGY']
{'STATUS__TP_LONG_30': {'Category': 'Fixed TP/SL', 'TP_Value': 5, 'SL_Value': 10}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,1.387651e+09,0.774627,670
1,LEADING_ONLY,1.451231e-02,0.491054,4024
2,TP_SL_LEADING,3.849839e-01,0.494533,4024


In [54]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#BTCUSDT -- 4H	STRATEGY_NBR_3544	29	            506,599 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 0

STRATEGY_USE                        = 'STRATEGY_NBR_3544'
TP_SL_USE                           = '29'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_PSAR']
{'STATUS__TP_LONG_29': {'Category': 'Fixed TP/SL', 'TP_Value': 5, 'SL_Value': 5}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,1.802972e+06,0.608333,480
1,LEADING_ONLY,1.030509e-08,0.494688,6966
2,TP_SL_LEADING,2.064191e-08,0.494688,6966


In [38]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#DOGEUSDT -- 4H	STRATEGY_NBR_38	    30	            1,038,485,750 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 3

STRATEGY_USE                        = 'STRATEGY_NBR_40'
TP_SL_USE                           = '30'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_MACD_ZERO_LINE_STRATEGY']
{'STATUS__TP_LONG_30': {'Category': 'Fixed TP/SL', 'TP_Value': 5, 'SL_Value': 10}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,1.038486e+09,0.761378,813
1,LEADING_ONLY,4.489722e-05,0.486649,6966
2,TP_SL_LEADING,1.741147e-03,0.488946,6966


In [31]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#AVAXUSDT -- 4H	STRATEGY_NBR_4	    34	            97,732 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 5

STRATEGY_USE                        = 'STRATEGY_NBR_4'
TP_SL_USE                           = '34'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_EMA_4__EMA_24']
{'STATUS__TP_LONG_34': {'Category': 'Fixed TP/SL', 'TP_Value': 10, 'SL_Value': 2.5}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,596610.220260,0.324219,256
1,LEADING_ONLY,8410.267504,0.351585,347
2,TP_SL_LEADING,442754.827526,0.322767,347


In [56]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#BTCUSDT -- 4H	STRATEGY_NBR_3544	RL_STD_12_2	    341,344 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 0

STRATEGY_USE                        = 'STRATEGY_NBR_3544'
TP_SL_USE                           = 'RL_STD_12_2'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_PSAR']
{'STATUS__TP_LONG_RL_STD_12_2': {'Category': 'Std Dev TP/SL', 'Risk_Level': 1.5, 'Reward_Factor': 2, 'Std_Window': 20}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,1.531121e+06,0.533970,942
1,LEADING_ONLY,1.030509e-08,0.494688,6966
2,TP_SL_LEADING,4.927833e-09,0.490956,6966


In [61]:
#FILE	        STRATEGY	        TP_SL	        END_BALANCE__1 
#ADAUSDT -- 4H	STRATEGY_NBR_197	28	            2,876 


TIMEFRAME_RD                        = ['1D', '4H'] 
COINS                               = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT']
T                                   = 1
C                                   = 4

STRATEGY_USE                        = 'STRATEGY_NBR_197'
TP_SL_USE                           = '28'

print(STRATEGY_NAME)
print(IMPLEMENTED_PROFIT)

BACKTEST_OUTPUT.API_DATA

['STRATEGY_EMA_8__EMA_18', 'STRATEGY_MACD_SIGNAL_LINE']
{'STATUS__TP_LONG_28': {'Category': 'Fixed TP/SL', 'TP_Value': 5, 'SL_Value': 2.5}}


,METHOD,END_BALANCE,WIN_RATE,NBR_TRADES
0,TP_SL_ONLY,2876.470604,0.442553,235
1,LEADING_ONLY,8199.453470,0.399267,273
2,TP_SL_LEADING,3743.069607,0.443223,273
